# Problem 4 : Estimation - BLP

In [27]:
using Plots, DataFrames, CSV, GLM
using Optim, Distributions, Random, ForwardDiff
using LinearAlgebra,StatsFuns

Random.seed!(6789998212);

In [28]:
# load in csv
df = DataFrame(CSV.File("../data/ps1_ex4.csv"));

In [29]:
# simulate individual taste shocks from N(μ,Σ)
draw_sim = function(μ, Σ, N) # return N x L matrix
    # draw shocks
    v = rand(MvNormal(μ, Σ), N)
    
    return v
end

#11 (generic function with 1 method)

In [30]:
# get data
n_markets = 5
n_sim = 50

x_t = []
for t in 1:n_markets 
   push!(x_t, Array(df[df[!,:market].==t, [:p, :x]]))
end

s_t = []
for t in 1:n_markets 
    push!(s_t, Array(df[df[!,:market].==t, [:shares]]))
end

z_t = []
for t in 1:n_markets 
    push!(z_t, Array(df[df[!,:market].==t, [:z1, :z2, :z3, :z4, :z5, :z6, :x]]))
end

x_jt = Array(df[df[!,:market] .<= n_markets,[:p, :x]]);

z_jt = Array(df[df[!,:market] .<= n_markets,[:z1, :z2, :z3, :z4, :z5, :z6, :x]]);

v = draw_sim([0;0], [1 0;0 1], n_sim)

2×50 Matrix{Float64}:
 0.123561  -0.201126  0.780628  -0.0984454  …  -1.41657  -0.895814  -2.1186
 0.805005  -0.780405  0.901409  -2.97667        1.01271  -0.746068  -0.440065

# Part 1: BLP

## Inner loop
`get_shares` calculates the shares of each product in a particular market $t$. $\delta$ should be a vector of length $J$; $x$ should be a matrix of size $J \times 2$; and $v$ should be a vector of length $L$.

`delta_contraction` iterates the $\delta_{jt}$ in a particular market $t$. $\delta$ should be a vector of length $J$; $x$ should be a vector of characteristics with length $J$; $s$ should be a vector of observed shares with length $J$; $v$ should be a vector of length $L$. 

`market_iterate` performs the contraction over each $t$ markets, it recoves $\delta_{jt}$, which is a vector of length $J \times T$.

In [7]:
# get shares in a market given some fixed gamma and delta
get_shares = function(δ, Γ, x, v)
    # we want to get share_{jt} using simulated values of $v_i$ (drawn above)
    # shares should be vector of length J
    numerator = exp.(δ .+ x * Γ * v)
    adj = maximum(numerator, dims = 1)
    denominator = sum((numerator ./ adj), dims = 1) .+ (1 ./ adj)
    shares = sum((numerator ./ adj) ./ denominator, dims = 2) ./ size(v)[2]
    
    return shares
end

# inner loop: contraction to find δ
delta_contraction = function(δ₀, Γ, s, x, v, tol = 1e-12, max_iter = nothing)

    # here δ is a vector of length J
    δ = δ₀
    err = 1000
    n = 0
    maxed_iter = false
    
    while (err > tol) && (maxed_iter === false)
        δ_old = δ
        
        # update delta
        δ = δ_old + log.(s) - log.(get_shares(δ_old, Γ, x, v))
        
        # difference 
        err = maximum(abs.(δ - δ_old)) 
        
        # (optional) max iterations block
        n += 1
        if max_iter !== nothing
            maxed_iter = (n == max_iter)
        end
    end
    
    return δ
end

# iterate over each market
market_iterate = function(Γ, s_t, x_t, v, tol = 1e-12, max_iter = nothing)
   
    δ = []
    for t in 1:size(s_t)[1]
        s = s_t[t]
        x = x_t[t]
        δ₀ = ones(size(s)[1])
        push!(δ, delta_contraction(δ₀, Γ, s, x, v, tol, max_iter) ) 
    end
    return δ
end

#7 (generic function with 3 methods)

## Outer loop
`residuals` does IV-GMM using the provided weighting matrix. z_jt should be a matrix of $Z$ excluded and included intruments of size $TJ \times Z$. Returns linear parameters (vector of length $2$) and $\xi_{jt}$ residuals (vector of length $J \times T$)

`gmm_objective` Reads in $TJ$-length vector $x$_jt and $TJ \times Z$ matrix $z$_jt. Calculates sample moments (size of instrument vector, $Z$) and optimal weighting matrix ($Z \times Z$). Returns scalar objective and matrix.

## Gradient
Estimator is $$\nabla G(\theta) = 2(Z'J_\theta)'W(Z'\xi(\theta))$$
-$Z$ is $JT \times Z$ matrix if instruments

-$\xi$ is $JT \times 1$ matrix of unobserved mean utilities

-$W$ is $Z \times Z$ 

$$ W = \left[(\xi(\theta) \circ Z)' (\xi(\theta) \circ Z) \right]^{-1}$$

-$J_\theta$ is $JT \times 3$

$$ J_\theta = -f_\xi^{-1} f_\theta$$

- For each t, $f_\xi$ is a $J \times J$ matrix: $\left\{\frac{\partial s_{ij}}{\partial \xi_k}\right\}_{j,k}$

$$ \frac{\partial s_{ij}}{\partial \xi_k} = -s_{ij}s_{ik}, \quad \frac{\partial s_{ij}}{\partial \xi_j} = s_{ij}(1-s_{ij}) $$

- For each t, $f_\theta$ is a $J \times 3$ matrix: 
$$\begin{bmatrix} s_{ij}s_{i0}\left(p_j - \sum_k s_{ik}p_k \right)\nu_{1i}  &  s_{ij}s_{i0}\left(x_j - \sum_k s_{ik}x_k \right)\nu_{1i} & s_{ij}s_{i0}\left(x_j - \sum_k s_{ik}x_k \right)\nu_{2i} \end{bmatrix}_{j} $$

All matrices are stacked $J \times \cdot$ over $T$ markets

In [116]:
# returns residuals for a given δ, estimates linear parameters given instruments
resid = function(δ_jt, x_jt, z_jt, W)
    # iv-gmm
    θ₁ = inv(x_jt' * z_jt * W * z_jt' * x_jt) * (x_jt' * z_jt * W * z_jt' * δ_jt)
    ξ_jt = δ_jt - x_jt * θ₁
    
    return ξ_jt, θ₁ 
    
end

# calculates gmm objective for outer loop
function gmm_objective(ξ_jt, z_jt, W)   
    # empirical moments, weighting matrix
    g = (ξ_jt' * z_jt) / size(ξ_jt)[1] 
    
    # gmm objective
    G = g * W * g'
    
    return G
end

# performs outer loop
function outer_loop(θ₂, s_t, x_t, x_jt, z_jt, v, W, tol = 1e-12, max_iter = nothing)
    # Pass through guess
    @time Γ = [θ₂[1] 0 ; θ₂[2] θ₂[3]] # lower triangular
    
    # Perform inner loop
    @time δ = market_iterate(Γ, s_t, x_t, v, tol, max_iter)
    
    # convert to JT x 1 (stacked J x 1 vectors for each t)
    δ_jt = vec(reduce(hcat,δ)) 
    
    # intermediate step
    @time ξ_jt, θ₁ = resid(δ_jt, x_jt, z_jt, W)
    
    # gmm step
    @time G = gmm_objective(ξ_jt, z_jt, W)
    
    println(G)
    
    return G
end

# Steps to calculate gradient...
# have data s_t, x_t, z_t
# first step will return θ           -> Γ  x
# 1. run market_iterate() with Γ     -> δ  x
# 2. run resid() with δ              -> ξ x
# 3. calculate W with ξ and Z        -> W x
# 4. calculate J with δ and Γ*       -> J  x
# *(for each i, calculate s_ij vector, do elementwise mult with p_j, v, and sum to get f_xi loop through j,k for f_theta)
# 5. run gradient() with J, W, ξ, Z  -> ∇

# helper function in gradient call: for each market get Jacobian of ξ(θ)
function jacobian_xi(δ, Γ, x, v)
    # need individual shares
    numerator = exp.(δ .+ x * Γ * v)
    adj = maximum(numerator, dims = 1)
    denominator = sum((numerator ./ adj), dims = 1) .+ (1 ./ adj)
    shares = (numerator ./ adj) ./ denominator # J x L
    
    # calculate partials of f(θ) = s - S(ξ,θ), denoted fξ and fθ
    fξ_store = []
    fθ_store = [] 
    for i = 1:size(v)[2]
        s_i = shares[:,i]
        s_i0 = 1 - sum(s_i)
        v_i = v[:,i]
        
        fξ_i = s_i * s_i' + diagm(s_i)
        
        fθ₁ = s_i .* (x[:,1] .- (s_i0 .* (s_i' * x[:,1]))) .* v[1]
        fθ₂ = s_i .* (x[:,2] .- (s_i0 .* (s_i' * x[:,2]))) .* v[1]
        fθ₃ = s_i .* (x[:,2] .- (s_i0 .* (s_i' * x[:,2]))) .* v[2]
        fθ_i = hcat(fθ₁, fθ₂, fθ₃)
        
        push!(fξ_store, fξ_i)
        push!(fθ_store, fθ_i)        
    end
    
    # calculate Jacobian
    J = -1 .* inv(mean(fξ_store)) * mean(fθ_store)
   
    return J
end
    
function gmm_gradient!(θ₂, s_t, x_t, x_jt, z_jt, v, W, ∇, tol = 1e-12, max_iter = nothing)
    # Pass through guess
    Γ = [θ₂[1] 0 ; θ₂[2] θ₂[3]] # lower triangular
    
    # Recover model objects from estimates parameters: Γ, and data: s_t, x_t, z_t, and v (simulated)    
    # δ(θ)
    δ = market_iterate(Γ, s_t, x_t, v, tol, max_iter)
    
    # ξ(θ)
    δ_jt = vec(reduce(hcat,δ)) 
    ξ_jt = resid(δ_jt, x_jt, z_jt, W)[1]
    ξ_t = reshape(ξ_jt, 6, Int64(size(ξ_jt)[1] / 6))
    
    # Analytic matrices
    # Jacobian
    J_t = []
    for t = 1:size(x_t)[1]
        push!(J_t, jacobian_xi(δ[t], Γ, x_t[t], v))
    end
    # J = reduce(vcat, J_t) # flatten to JT x 3 matrix
    
    # Weighting (note: put outside, we want to fix W through run)
    # W = inv((z_jt .* ξ_jt)' * (z_jt .* ξ_jt)) * size(ξ_jt)[1]
    
    # Calculate gradient
    ∇ = []
    for t in 1:size(s_t)[1]
        ∇_t = 2 .* (z_t[t]' * J_t[t])' * W * (z_t[t]' * ξ_t[:,t])
        push!(∇, ∇_t)
    end

    return mean(∇)
end

gmm_gradient! (generic function with 3 methods)

In [74]:
δ₀ = ones(6)
params0 = ones(3)
#params0 = [2.1095000932155177, 0.026342237451039658, 0.029950456541307163]
Γ = [params0[1] 0 ; params0[2] params0[3]]

w = inv(z_jt' * z_jt) * size(z_jt)[1];

tol = 1e-12;
max_iter = 100;

f(θ₂) = outer_loop(θ₂, s_t, x_t, x_jt, z_jt, v, w, tol, max_iter)
g!(∇,θ₂) = gmm_gradient!(θ₂, s_t, x_t, x_jt, z_jt, v, w, ∇, tol, max_iter)

g! (generic function with 1 method)

In [118]:
# @time o = Optim.optimize(f, g!,params0, LBFGS())
@time o = Optim.optimize(f, g!, params0, LBFGS(), Optim.Options(show_trace = true, show_every = 10))

  0.000006 seconds (12 allocations: 432 bytes)
  0.005105 seconds (14.52 k allocations: 6.234 MiB)
  0.000023 seconds (18 allocations: 3.984 KiB)
  0.006866 seconds (3.83 k allocations: 195.924 KiB, 99.61% compilation time)
2.1714405022761856
Iter     Function value   Gradient norm 
     0     2.171441e+00              NaN
 * time: 0.0
  0.076665 seconds (97.18 k allocations: 16.837 MiB, 84.55% compilation time)


 * Status: failure

 * Candidate solution
    Final objective value:     2.171441e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    0
    f(x) calls:    1
    ∇f(x) calls:   1


In [ ]:
J_t = []
for t in 1:5
    push!(J_t, jacobian(delt[1], Γ, x_t[1], v))
end

J_t[1]
reduce(vcat, J_t)

In [117]:
∇ = ones(3)
gmm_gradient!(θ₂, s_t, x_t, x_jt, z_jt, v, w, ∇, tol, max_iter)

3-element Vector{Float64}:
   0.17312587104225247
  -2.4417173794561013
 -15.907888393485962

In [51]:
w = z_jt' * z_jt
ξ = resid(δ_jt, x_jt, z_jt, w)[1]

inv((z_jt .* ξ)' * (z_jt .* ξ)) * size(ξ)[1]


7×7 Matrix{Float64}:
  0.00482719   -0.0027584    -0.00201489   …   0.000947291  -0.00198364
 -0.0027584     0.0035374    -0.000299187     -0.00156654    0.00184279
 -0.00201489   -0.000299187   0.00328508       0.000500983   0.000175094
 -0.00217568    0.000587866   0.000929431      0.000803485   0.000566466
  0.000885222  -0.000124342  -0.00106843       2.38406e-5   -0.00044566
  0.000947291  -0.00156654    0.000500983  …   0.00334798   -0.00137792
 -0.00198364    0.00184279    0.000175094     -0.00137792    0.00291332

In [ ]:
g, w = outer_loop([1 1 1], s_t, x_t, x_jt, z_jt, v)

In [ ]:
delt = market_iterate(Γ, s_t, x_t, v, 1e-1, 3)
delt[1]
get_shares(delt[1], Γ, x_t[1], v)[1]

In [77]:
θ₂ = o.minimizer
Γ = [θ₂[1] 0 ; θ₂[2] θ₂[3]]

δ = market_iterate(Γ, s_t, x_t, v)
    
δ_jt = vec(reduce(hcat,δ)) 
ξ_jt, θ₁ = resid(δ_jt, x_jt, z_jt, w)
    
S_t = []
for t in 1:size(x_t)[1]
    push!(S_t, get_shares(δ[t], Γ, x_t[t], v))
end
S = mean(reduce(hcat,S_t), dims = 2)

s = mean(reduce(hcat,s_t), dims = 2)

print(θ₁, θ₂)
maximum(reduce(hcat,S_t) - reduce(hcat,s_t))


reshape(ξ_jt,6,Int64(size(ξ_jt)[1] / 6))

[-0.7667679910719568, 0.397364825136542][2.1134352999771044, 0.02706264320005767, 0.029948155059233695]

6×5 Matrix{Float64}:
 -3.48201    -4.10896   -2.56582   -0.0305048  -0.931278
  0.0672305  -2.54388   -3.83699   -1.69399    -1.96049
 -1.67977     1.39517   -3.05901   -2.57768     0.0758904
 -1.54805    -2.25454   -3.29644   -4.81321    -3.62375
  1.36491    -1.09951   -0.751778  -1.10928    -0.579442
 -0.398699    0.766565  -0.944682   1.17632    -1.46153